In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.metrics import classification_report, confusion_matrix

print(tf.__version__)

In [22]:
tf.test.is_gpu_available()

In [23]:
train_path = '../input/satellite-images-of-hurricane-damage/train_another/'
val_path = '../input/satellite-images-of-hurricane-damage/validation_another/'

In [24]:
categories = glob(train_path + '*')

In [25]:
categories

In [26]:
print(len(glob(categories[0] + '/*')))
print(len(glob(categories[1] + '/*')))

In [27]:
IMAGE_SIZE = [224,224]
model = VGG16(include_top = False,input_shape = IMAGE_SIZE + [3],weights = 'imagenet')
for layers in model.layers:
    layers.trainable = False

In [28]:
model.summary()

In [29]:
#modifying the model
x = tf.keras.layers.Flatten()(model.output)
# x = tf.keras.layers.Dense(units = 4096,activation = 'relu')(x)
x = tf.keras.layers.Dense(units = len(categories),activation = 'sigmoid')(x)

In [30]:
model = tf.keras.Model(inputs = model.input,outputs = x)
model.summary()

In [31]:
#Hyperparameters
lr = 0.001
batch_size = 32


loss_fn = tf.keras.losses.BinaryCrossentropy()
optimiser = tf.keras.optimizers.Adam(learning_rate = lr)
model.compile(loss = loss_fn,optimizer = optimiser,metrics = ['accuracy','Recall','Precision'])

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip = True,
                                                                  rescale = 1./255,
                                                                 shear_range = 0.2,
                                                                 zoom_range = 0.2)
train_ds = train_generator.flow_from_directory(train_path,
                                               class_mode = 'categorical',
                                               target_size = (224,224),
                                               batch_size = batch_size)
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_ds = val_generator.flow_from_directory(val_path,
                                          class_mode = 'categorical',
                                          target_size = (224,224),
                                          shuffle = False,
                                          batch_size = batch_size)

In [32]:
img = next(train_ds)[0]
for i in range(2):
    plt.imshow(img[i])
    plt.title(img.shape)
    plt.show()

In [33]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 3,min_delta = 0.005)

history = model.fit(train_ds,
                    validation_data=val_ds,
                    steps_per_epoch = len(train_ds),
                    validation_steps = len(val_ds), 
                    epochs = 30,
                    callbacks = [callback])

In [72]:
obj = history.history
figure = plt.figure(figsize=(10, 7))
plt.plot(obj['accuracy'])
plt.plot(obj['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy','Validation Accuracy'],loc = 'lower right')
plt.show()

figure = plt.figure(figsize=(10, 7))
plt.plot(obj['loss'])
plt.plot(obj['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend(['Training Loss','Validation Loss'],loc = 'upper right')
plt.show()

In [35]:
y_pred = model.evaluate(val_ds,return_dict = True) 

In [36]:
y_pred

In [37]:
f1_score = 2*(y_pred['recall']*y_pred['precision'])/(y_pred['recall'] + y_pred['precision'])

In [38]:
f1_score

In [39]:
y_pred_1 = model.predict(val_ds)
y_true = val_ds.labels

In [52]:
print(classification_report(y_true,np.argmax(y_pred_1,axis = 1)))

In [69]:
classes=['Damage','No Damage']
con_mat_df = pd.DataFrame(confusion_matrix(y_true,np.argmax(y_pred_1,axis = 1)),
                     index = classes, 
                     columns = classes)

figure = plt.figure(figsize=(10, 7))
sns.heatmap(con_mat_df, annot=True,cmap='tab10',fmt='d')
plt.tight_layout()
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()